In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from model_config import ModelConfig
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20}) # set plot font sizes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
transition_window =10

warmup_start = '20210421'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

# Learning rate
learning_rate = 0.1

In [5]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)
vax_statuses = [Vax.yes, Vax.no]
x_train = tf.cast(df.loc[train_start:train_end,'Rt'].values, dtype=tf.float32)
x_test = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)


y_train = {}
y_train['G_in'] = tf.cast(df.loc[train_start:train_end,'general_ward_in'], dtype=tf.float32)
y_train['G_count'] = tf.cast(df.loc[train_start:train_end,'general_ward_count'], dtype=tf.float32)
y_train['I_count'] = tf.cast(df.loc[train_start:train_end,'icu_count'], dtype=tf.float32)
y_train['D_in'] = tf.cast(df.loc[train_start:train_end,'deaths_covid'], dtype=tf.float32) + 1

y_test = {}
y_test['G_in'] = tf.cast(df.loc[train_start:test_end,'general_ward_in'], dtype=tf.float32)
y_test['G_count'] = tf.cast(df.loc[train_start:test_end,'general_ward_count'], dtype=tf.float32)
y_test['I_count'] = tf.cast(df.loc[train_start:test_end,'icu_count'], dtype=tf.float32)
y_test['D_in'] = tf.cast(df.loc[train_start:test_end,'deaths_covid'], dtype=tf.float32)

2022-02-27 18:02:25.252085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-27 18:02:25.252585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-27 18:02:25.252624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-27 18:02:25.252652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-27 18:02:25.252695: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [7]:
config = ModelConfig.from_json('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/model_config.json')

In [8]:
model = CovidModel([Vax.no, Vax.yes], [Comp.A, Comp.M, Comp.G, Comp.GR, Comp.I, Comp.IR, Comp.D],
                 transition_window,
                config, posterior_samples=1000, debug_disable_theta=False)

pre_training_preds=model.call(x_train)

In [6]:
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate,#beta_1=0.1, beta_2=0.1
)
#loss((y_train['G_count'], y_train['G_in'], y_train['I_count'], y_train['D_in']), pre_training_preds)

In [7]:
logging_callbacks = get_logging_callbacks('/mnt/c/Users/kheut/logs/covid/linear_warm_fixed_loss_2')

In [ ]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([(y_train['G_count'], y_train['G_in'], y_train['I_count'], y_train['D_in'])]),
         epochs=2000, batch_size=0,
        callbacks=logging_callbacks)

Epoch 1/2000
G count: -8096.701171875
G in: -1721.12451171875
I count: -1206.6898193359375
D in: -12.733940124511719


2022-02-26 22:16:22.430136: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


1/1 [==============================] - 46s 46s/step - loss: 37931.9453
Epoch 2/2000
G count: -5474.2451171875
G in: -1052.27392578125
I count: -1014.6215209960938
D in: -10.86467456817627
1/1 [==============================] - 45s 45s/step - loss: 32647.4668
Epoch 3/2000
G count: -4216.8779296875
G in: -719.3932495117188
I count: -989.9917602539062
D in: -10.20695686340332
1/1 [==============================] - 44s 44s/step - loss: 30704.1250
Epoch 4/2000
G count: -3456.790771484375
G in: -535.3489990234375
I count: -881.9038696289062
D in: -9.569940567016602
1/1 [==============================] - 43s 43s/step - loss: 28444.0820
Epoch 5/2000
G count: -3420.008056640625
G in: -494.86395263671875
I count: -946.8333129882812
D in: -9.664220809936523
1/1 [==============================] - 44s 44s/step - loss: 28044.7422
Epoch 6/2000
G count: -2839.205078125
G in: -354.6944885253906
I count: -935.1959838867188
D in: -9.507257461547852
1/1 [==============================] - 44s 44s/step - lo

In [1]:
model.variables

NameError: name 'model' is not defined

In [ ]:
config.rho_M.value

In [9]:
model.unconstrained_rho_M

{0: {'loc': <tf.Variable 'rho_M_loc_0:0' shape=() dtype=float32, numpy=0.0>,
  'scale': <tf.Variable 'rho_M_scale_0:0' shape=() dtype=float32, numpy=-2.2521684>},
 1: {'loc': <tf.Variable 'rho_M_loc_1:0' shape=() dtype=float32, numpy=-0.20067078>,
  'scale': <tf.Variable 'rho_M_scale_1:0' shape=() dtype=float32, numpy=-2.2521684>}}

In [ ]:
model._constrain_parameters()

In [ ]:
model._sample_and_reparameterize()

In [ ]:
model.rho_M_params

In [ ]:
model.rho_M_samples_constrained

In [ ]:
model.warmup_A_params